In [ ]:
import pandas as pd
import numpy as np
import torch
from torch import nn
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from __future__ import print_function
import torch.nn.functional as F
from torch.autograd import Variable
import torch.utils.data as data_utils

In [ ]:
matplotlib.__version__

In [ ]:
df_testt = pd.read_csv("14_15.csv")

In [ ]:
df_testt

In [ ]:
df_train = pd.read_csv("Train_14_18.csv")
df_train_backup=df_train.copy()
print(df_train)
df_train.fillna(0, inplace = True)

In [ ]:
df_train.info()

In [ ]:
df_valid = pd.read_csv("Valid_18_20.csv")
df_valid_backup=df_valid.copy()
print(df_valid)
df_valid.fillna(0, inplace = True)

In [ ]:
df_valid.info()

In [ ]:
df_test = pd.read_csv("Test_20_22.csv")
df_test_backup=df_test.copy()
df_test.fillna(0, inplace = True)
df_test

In [ ]:
df_test.info()

Transfer catagorial to int

In [ ]:
df_train.dtypes 

All numbers, no need,

In [ ]:
y_train = df_train['W-L%']
x_train = df_train.drop(['School', 'W', 'L', 'Rk','Year','W-L%'], axis=1)
x_train.info()

In [ ]:
y_test = df_test['W-L%']
x_test = df_test.drop(['School', 'W', 'L', 'Rk','Year','W-L%'], axis=1)
x_test.info()

In [ ]:
y_valid = df_valid['W-L%']
x_valid = df_valid.drop(['School', 'W', 'L', 'Rk','Year','W-L%'], axis=1)
x_valid.info()

In [ ]:
x_train_array = np.array(x_train)
y_train_array = np.array(y_train)
x_valid_array = np.array(x_valid)
y_valid_array = np.array(y_valid)
x_test_array = np.array(x_test)
y_test_array = np.array(y_test)
x_train_array

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
def plot_predictions(train_data=x_train_array, 
                     train_labels=y_train_array, 
                     test_data=x_valid_array, 
                     test_labels=y_valid_array, 
                     predictions=None):
  """
  Plots training data, test data and compares predictions.
  """
  plt.figure(figsize=(10, 7))

  # Plot training data in blue
  plt.scatter(train_data, train_labels, c="b", s=4, label="Training data")
  
  # Plot test data in green
  plt.scatter(test_data, test_labels, c="g", s=4, label="Testing data")

  if predictions is not None:
    # Plot the predictions in red (predictions were made on the test data)
    plt.scatter(test_data, predictions, c="r", s=4, label="Predictions")

  # Show the legend
  plt.legend(prop={"size": 14});

In [ ]:
# Create a Linear Regression model class
class LinearRegressionModel(nn.Module): # <- almost everything in PyTorch is a nn.Module (think of this as neural network lego blocks)
    def __init__(self):
        super().__init__() 
        self.weights = nn.Parameter(torch.randn(1, # <- start with random weights (this will get adjusted as the model learns)
            requires_grad=True, # <- can we update this value with gradient descent?
            dtype=torch.float # <- PyTorch loves float32 by default
        ))

        self.bias = nn.Parameter(torch.randn(1, # <- start with random bias (this will get adjusted as the model learns)
            requires_grad=True, # <- can we update this value with gradient descent?
            dtype=torch.float # <- PyTorch loves float32 by default
        ))

    # Forward defines the computation in the model
    def forward(self, x: torch.Tensor) -> torch.Tensor: # <- "x" is the input data (e.g. training/testing features)
        return self.weights * x + self.bias # <- this is the linear regression formula (y = m*x + b)

In [ ]:
# Set manual seed since nn.Parameter are randomly initialzied
torch.manual_seed(42)

# Create an instance of the model (this is a subclass of nn.Module that contains nn.Parameter(s))
model_0 = LinearRegressionModel()

# Check the nn.Parameter(s) within the nn.Module subclass we created
list(model_0.parameters())

In [ ]:
model_0.state_dict()

In [ ]:
with torch.inference_mode(): 
    y_preds = model_0(x_valid_array)

In [ ]:
print(f"Number of testing samples: {len(x_valid_array)}") 
print(f"Number of predictions made: {len(y_preds)}")
print(f"Predicted values:\n{y_preds}")

In [ ]:
# Create the loss function
loss_fn = nn.L1Loss() # MAE loss is same as L1Loss
# Create the optimizer
optimizer = torch.optim.SGD(params=model_0.parameters(), # parameters of target model to optimize
                            lr=0.01) # learning rate (how much the optimizer should change parameters at each step, higher=more (less stable), lower=less (might take a long time))

In [ ]:
plot_predictions(predictions=y_preds)

In [ ]:
torch.manual_seed(42)

# Set the number of epochs (how many times the model will pass over the training data)
epochs = 100

# Create empty loss lists to track values
train_loss_values = []
test_loss_values = []
epoch_count = []

for epoch in range(epochs):
    ### Training

    # Put model in training mode (this is the default state of a model)
    model_0.train()

    # 1. Forward pass on train data using the forward() method inside 
    y_pred = model_0(x_train_array)
    # print(y_pred)

    # 2. Calculate the loss (how different are our models predictions to the ground truth)
    loss = loss_fn(y_pred, y_train_array)

    # 3. Zero grad of the optimizer
    optimizer.zero_grad()

    # 4. Loss backwards
    loss.backward()

    # 5. Progress the optimizer
    optimizer.step()

    ### Testing

    # Put the model in evaluation mode
    model_0.eval()

    with torch.inference_mode():
      # 1. Forward pass on test data
      test_pred = model_0(x_valid_array)

      # 2. Caculate loss on test data
      test_loss = loss_fn(test_pred, y_valid_array.type(torch.float)) # predictions come in torch.float datatype, so comparisons need to be done with tensors of the same type

      # Print out what's happening
      if epoch % 10 == 0:
            epoch_count.append(epoch)
            train_loss_values.append(loss.detach().numpy())
            test_loss_values.append(test_loss.detach().numpy())
            print(f"Epoch: {epoch} | MAE Train Loss: {loss} | MAE Test Loss: {test_loss} ")

In [ ]:
# Plot the loss curves
plt.plot(epoch_count, train_loss_values, label="Train loss")
plt.plot(epoch_count, test_loss_values, label="Test loss")
plt.title("Training and test loss curves")
plt.ylabel("Loss")
plt.xlabel("Epochs")
plt.legend();

In [ ]:
# Find our model's learned parameters
print("The model learned the following values for weights and bias:")
print(model_0.state_dict())
print("\nAnd the original values for weights and bias are:")
print(f"weights: {weight}, bias: {bias}")

In [ ]:
# 1. Set the model in evaluation mode
model_0.eval()

# 2. Setup the inference mode context manager
with torch.inference_mode():
  # 3. Make sure the calculations are done with the model and data on the same device
  # in our case, we haven't setup device-agnostic code yet so our data and model are
  # on the CPU by default.
  # model_0.to(device)
  # X_test = X_test.to(device)
  y_preds = model_0(X_test)
y_preds

In [ ]:
plot_predictions(predictions=y_preds)

In [ ]:
# 1. Put the loaded model into evaluation mode
loaded_model_0.eval()

# 2. Use the inference mode context manager to make predictions
with torch.inference_mode():
    loaded_model_preds = loaded_model_0(x_valid_array) # perform a forward pass on the test data with the loaded model

In [ ]:
y_preds == loaded_model_preds